# Shor's algorithm

Shor's algorithm, named after mathematician Peter Shor, is a quantum algorithm (an algorithm that runs on a quantum computer) for integer factorization, formulated in 1994. Informally, it solves the following problem: Given an integer **N**, find its prime factors.

## Overview

*Lemma*: Factoring is equivalent to finding a nontrivial squareroot of 1 mod N.

**Complexity:**

- Complexity on quantum computer: 
![quantum_complexity](https://wikimedia.org/api/rest_v1/media/math/render/svg/41344b55ef5e6494bb835b1adde9a4be4403303c)

- Complexity on classical computer:
![classical_complexity](https://wikimedia.org/api/rest_v1/media/math/render/svg/1956460a1cea4f2794fa0487f886e2a64cedc244)

![complexity_graph](https://dal.objectstorage.open.softlayer.com/v1/AUTH_039c3bf6e6e54d76b8e66152e2f87877/images-classroom/shor-figure1l0qpbqeb138fr.png)

**Steps:**

- 2001 ==> *IBM factored 15 using 7 qubits*
- 2012 ==> *15 factorized with solid-state qubits and 21 factorized*
- 2012 ==> *143 factorized using adiabatic quantum computation (rather than Shor's algorithm)*
- 2014 ==> *adiabatic quantum computation factorized 56153*

## Structure of the algorithm

![process](https://algassert.com/assets/2017-08-28-shor-algorithm/flowchart.png)

Shor's algorithm consists of two parts:

**1) [CLASSICAL PART]** A reduction, which can be done on a classical computer, of the factoring problem to the problem of order-finding.

**2) [QUANTUM PART]** A quantum algorithm to solve the order-finding problem.


Shor's algorithm hinges on a result from number theory. This result is:

*The function $ \mathcal {F}$(a) = x^a mod n is a periodic function, where x is an integer coprime to n. In the context of Shor's algorithm n will be the number we wish to factor. When two numbers are coprime it means that their greatest common divisor is 1.*

The reason why this function is of utility in factoring large numbers is the following:

since $ \mathcal {F}$(a) is a periodic function, it has some period r. We know that x^0 mod n = 1, so x^r mod n = 1, and x^(2r) mod n = 1 and so on since the function is periodic.

![algebraic_manipulation](http://mathurl.com/y7xnrfh8.png)

And if r is an even number (because **x^r - 1 = 0 mod N** means that N divides x^r - 1 and since N is odd, r must be even):

![even_number](http://mathurl.com/y9xnja7t.png)

This means that the product (x^r/2 -1)(x^r/2 + 1) is an integer multiple of n, the number to be factored. So long as x^r/2 is not equal to $ \pm$1, then at least one of (x^r/2 - 1) or (x^r/2 + 1) must have a nontrivial factor in common with n. So by computing gcd(x^r/2 - 1, n), and gcd(x^r/2 + 1, n), we will obtain a factor of n, where gcd is the greatest common denominator function.

### CLASSICAL PART

**Lemma:** Let N be an odd composite, with at least two distinct prime factors, and let x be uniformly random between 0 and N-1. if GCD(x, N) = 1, then with probability at least 1/2, the order r of x (mod N) is even, and x^r/2 is a nontrivial square root of 1 (mod N).

In [25]:
import math
import random
import sys
from fractions import Fraction
from builtins import input

def shor(N, attempts = 1):
    
    # build compilation engine list
    rule_set = DecompositionRuleSet(modules=[projectq.libs.math,
                                             projectq.setups.decompositions])
    compilerengines = [AutoReplacer(rule_set),
                       InstructionFilter(high_level_gates),
                       TagRemover(),
                       LocalOptimizer(3),
                       AutoReplacer(rule_set),
                       TagRemover(),
                       LocalOptimizer(3)]

    # make the compiler and run the circuit on the simulator backend
    eng = MainEngine(Simulator(), compilerengines)
    
    for attempt in range(attempts):
        print("-----------------------------------")
        print("\nAttempt #" + str(attempt))
        
        x = random.randint(0, N)
        print("Random x between 0 and N-1 --> ", str(x))
        
        '''If the GCD is not 1, x is a nontrivial factor of N, so we're done'''
        if (math.gcd(x, N) != 1):
            print("\nFactors found classically, re-attempt...")
            continue
        
        '''Otherwise it means that x and N are coprime
        Here the quantum part starts: Shor's algorithm tries to find r, the period of x^a mod n, where n is 
        the number to be factored and x is an integer coprime to n.
        It is important to underline the r  is the smallest positive integer such that x^r = 1 mod N
        '''
        r = find_period(eng, N, x, verbose=False)
        
        '''If r is odd or if x^r/2 = -1 (mod N), choose another x
        EXPLANATION: we already know that x^r/2 is NOT congruent to 1 (mod N), otherwise the order of x would be r/2,
        instead of r. So we have to check only that x^r/2 is NOT congruent to -1 (mod N)
        '''
        if ((r % 2 != 0) or (pow(x, int(r/2), N) == -1)): 
            print("r is odd or x^r/2 = -1 (mod N), re-attempt...")
            continue
            
        print("\nPeriod found: " + str(r))
        
        p = math.gcd(x**int(r / 2) + 1, N)
        q = math.gcd(x**int(r / 2) - 1, N)
        
        if ((not p * q == N) and p * q > 1 and int(1. * N / (p * q)) * p * q == N):
            p, q = p * q, int(N / (p * q))
        if p * q == N and p > 1 and q > 1:
            print("\nFactors found: {} * {} = {}.".format(p, q, N))
            break
        else:
            print("\nBad luck: Found {} and {}".format(p, q))

### QUANTUM PART

In the 1760s, Euler discovered a beautiful pattern to this period finding problem. Let n be the product of two prime numbers, p and q. Consider the sequence:

![sequence](https://s0.wp.com/latex.php?zoom=1.25&latex=x+%5Ctext%7B+mod+%7D+n%2C+x%5E2+%5Ctext%7B+mod+%7D+n%2C+x%5E3+%5Ctext%7B+mod+%7D+n%2C+x%5E4+%5Ctext%7B+mod+%7D+n%2C&bg=ffffff&fg=000&s=0)

If x is not divisible by p or q, then the above sequence will repeat with some period that evenly divides (p-1)(q-1).

#### Example
N = 21 (the number we want to factorize)<br>
x = 2 (chosen randomly)

<table style="border-collapse:collapse;border-spacing:0" class="tg"><tr><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:center;vertical-align:top">a</th><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">f(a) = x^a (mod N)</th></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">0</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">1</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">1</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">2</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">2</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">4</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">3</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">8</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">4</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">16</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">5</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">11</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">6</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">1</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">7</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">2</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">8</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">4</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">9</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">16</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">10</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">11</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">11</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">1</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">12</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:4px 20px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:left;vertical-align:top">2</td></tr></table>

And so on...

As we can see, the period r = 6 <br>
and if we are "lucky" ${(x^{(r/2)})^2 = 1 (mod N)}$ from which we can easily recover the factors of N.


#### Why do we need a quantum algorithm?
The problem is that the number of steps after which the sequence starts repeating itself could be almost as large as n (in RSA n is a very big number!)

That's why we use a quantum computer.

#### How does it work?
Shor's period-finding algorithm relies heavily on the ability of a quantum computer "to be in many states simultaneously". Physicists call this behavior a "superposition" of states. To compute the period of a function f, we evaluate the function at all points simultaneously.

But...

#### Two problems coming from the quantum theory
- Measurement: the state of a q-qubit quantum register is described by a vector of dimension 2^q, exponentially larger than the dimension of the vector required to describe q classical bits. However, there is a catch: in a classical computer we can simply read the state of the bits, whereas in a quantum computer we do not have direct, unrestricted access to the quantum state. Information on the quantum state are obtained by applying a measurement gate, after which the original quantum state is no longer available, since it collapses to a linear combination of only those basis states that are consistent with the outcome of the measurement.

- No cloning theorem: since measurement destroys the quantum state, it is natural to look for a way to create a copy of a quantum state, so that we can apply a measurement gate on it, leaving the original state untouched. Unfortunately, cloning is impossible. This is due to the fact that gates are unitary matrix [proof].

The Shor algorithm requires two quantum registers. At the beginning of the algorithm, one has to choose **q = 2^s** for some integer **s** such that **n^2 ≤ q < 2n^2**, where n is to be factored. More precisely, it proceeds as follows:

#### Phases
1) Create a quantum register and partition it into two parts, register 1 and register 2. Thus the state of our quantum computer is given by: |reg1, reg2>. 

Register 1 must have enough qubits to represent the integers as large as q - 1, while register 2 must have enough qubits to represent integers as large as n - 1 (the calculation for how many qubits are needed would be done on a classical computer).

2) Initialization: put register-1 in the uniform superposition, i.e. an equally weighted superposition of all integers from 0 to q - 1 (a superposition of the all the rows of the table shown above), and load register 2 with all zeros.

![state0](http://mathurl.com/y7gqualj.png)

3) Now apply the transformation x^a mod n to for each number stored in register 1 and store the result in register 2.
The new state is:

![state1](http://mathurl.com/y8e6atxu.png)


#### *Modular Exponentiation*
The bottleneck in the quantum factoring algorithm is modular exponentiation. The modular exponentiation problem, given n, x and r, find ${x^r (mod n)}$. Fundamentally, quantum modular exponentiation is ${O(n^3)}$; this means that the number of quantum gates or operations scales with the cube of the length in bits of the number to be factored. It consists in *2n* modular multiplications, each of which consists of ${O(n)}$ additions, each of which requires ${O(n)}$ operations. However, ${O(n^3)}$ operations do not necessarily require ${O(n^3)}$ time steps. On an abstract machine, it is relatively straightforward to see how to reduce each of those three layers to ${O(log n)}$ time steps, in exchange for more space and more total gates, giving a total running time of ${O(log^3 n)}$ if ${O(n^3)}$ qubits are available and an arbitrary number of gates can be executed concurrently on separate qubits. Such large numbers of qubits are not expected to be practical for the foreseeable future, that is why some people are trying to optimize this process (see [this paper for more info](https://arxiv.org/pdf/quant-ph/0408006.pdf)).

4) Measure the second register, and observe some value k. This has the side effect of collapsing register one into a equal superposition of each value a between 0 and q - 1 such that x^a mod n = k. The new state is:

![state2](http://mathurl.com/y9l64phf.png)

Where A is the set of *a*'s such that x^a mod n = k, and ||A|| is the number of elements in that set.

5) Compute the quantum Fourier transform on register one. 
The aim of applying QFT to the register-1 is to accumulate the wanted state |a, x^a (mod n) $\rangle$ such that it is possibly observed with significant probability.


#### *Discrete Fourier transform*
As the name indicates, DFT is the discrete version of Fourier transform. Most commonly, the input sequence is time series data or spatial samples, and the output sequence is frequency data, i.e., the Fourier spectrum.


#### *Quantum Fourier transform*
The quantum Fourier transform (for short: QFT) is a linear transformation on quantum bits, and is the quantum analogue of the discrete Fourier transform (see above). 

Like the classical Fourier transform, quantum Fourier transform (QFT) takes data from the original signal representation to the frequency domain representation. The QFT differs from the classical Fourier transform in that it operates on a superposition state and produces a different superposition state as the output.
**The point is: after applying the QFT, sampling the quantum computer's state can tell us what the dominant frequencies of the input state were (if any).**

The quantum Fourier transform can be performed efficiently on a quantum computer, with a particular decomposition into a product of simpler unitary matrices. Using a simple decomposition, the discrete Fourier transform on ${\displaystyle 2^{n}}$ amplitudes can be implemented as a quantum circuit consisting of only ${\displaystyle O(n^{2})}$ Hadamard gates and controlled phase shift gates, where ${\displaystyle n}$ is the number of qubits. This can be compared with the classical discrete Fourier transform, which takes ${\displaystyle O(n2^{n})}$ gates (where ${\displaystyle n}$ is the number of bits), which is exponentially more than ${\displaystyle O(n^{2})}$. However, the quantum Fourier transform acts on a quantum state, whereas the classical Fourier transform acts on a vector, so not every task that uses the classical Fourier transform can take advantage of this exponential speedup.


The QFT, when applied to a state |a${\rangle}$, changes it in the following manner:

![qft](http://mathurl.com/y9agjtb3.png)

So, the new state is:

![state3](http://mathurl.com/y8rt33cf.png)

6) Measure the state of register one, call this value m, this integer m has a very high probability of being a multiple of q/r, where r is the desired period.

7) At this point we repeat the algorithm to retrieve several distinct multiples of q/r. Once we have enough values, we can compute their GCD (with Euclid's algorithm) to retrieve q/r. q is given by the problem, so it is easy to compute r.

--------------------------------------------------------------------------------------------------------------------------------
**NOTE: The last point is true under one important assumption, i.e. that r divides q.<br>
Let's assume that q/r > 2r, meaning that the number of periods that we look at is comparable to and larger than the period itself.**

**Actually, the quantum circuit remains the same but let's suppose that at the end we measure a value L.<br>
The claim is that:
![claim](http://mathurl.com/ybx95am8.png)
for some integer t.**

- **L is the output of the algorithm**
- **q is kwnown**
- **r is the period we want to retrieve**
- **t is unknown**

**So how do we find both t and r?<br>
It turns out that t/r is the best approximation to L/q with a denominator as small as r, so we use the fact that r is much smaller than $\sqrt{q}$.<br>
Given that you can reconstruct t/r using a technique called continued fractions, which can be performed very quickly on a classical computer.**

#### Numerical example

We want to find the period of the function ${f(x) = x \mod 2}$.

It is easy to see that the period is 2.

We will use a 3-qubit system, so that q = 8. 

![example1](http://mathurl.com/yax4xflf.png)
![example2](http://mathurl.com/ybl6vuyt.png)

Now we measure |$f\rangle$. Then |$f\rangle$ will collapse into either |0$\rangle$ or |1$\rangle$

Let's suppose that our measurement returns $|f(x)\rangle$ = |1$\rangle$

![example3](http://mathurl.com/yaznw284.png)

Now we need to extract the period of the first register, applying again the Fourier transform

![example4](http://mathurl.com/yd7zpnbw.png)

Finally, if we take a few measurements we will be sure to measure both |$0\rangle$ and |$4\rangle$.

Therefore q/r = 4, and since q = 8, it is clear that r = 2.

## Circuit

**Circuit for period finding**
![period_finding](https://i.ibb.co/Ytg7PGt/period-finding.png)

**Circuit for factorization**
![factorization](https://i.ibb.co/7gb2KvJ/factoring.png)
It relies heavily on period finding, and so the circuit looks a lot like the circuit for period finding. The key difference is that we are finding the period of ${f(i) = x^i}$, and the number of bits we need to input is very large.

#### NOTE: 
**The implementation of the quantum gate *${Ux^i}$* as well as the inverse *QFT* gate are relatively complex and the right gate set to use is currently still an open question.**

In [18]:
import projectq.libs.math
import projectq.setups.decompositions
from projectq.backends import Simulator, ResourceCounter
from projectq.cengines import (AutoReplacer, DecompositionRuleSet,
                               InstructionFilter, LocalOptimizer,
                               MainEngine, TagRemover)
from projectq.libs.math import (AddConstant, AddConstantModN,
                                MultiplyByConstantModN)
from projectq.meta import Control
from projectq.ops import (All, BasicMathGate, get_inverse, H, Measure, QFT, R,
                          Swap, X)

In [3]:
'''As said in ProjectQ webpage --> "Simulating Shor s algorithm at the level of single-qubits gates and CNOTs already takes
quite a big of time for larger numbers than 15."
Returning TRUE allows to use the emulation feature, which does not decompose the modular arithmetic to low-level gates.'''
def high_level_gates(eng, cmd):
    return True

In [4]:
def find_period(eng, N, a, verbose=False):
    
    n = int(math.ceil(math.log(N, 2))) # log2(N)

    x = eng.allocate_qureg(n)

    '''Pauli-X gate: acts on a single qubit. It is the quantum equivalent of the NOT gate for classical computers'''
    X | x[0]

    measurements = [0] * (2 * n)  # will hold the 2n measurement results

    ctrl_qubit = eng.allocate_qubit()

    for k in range(2 * n):
        '''Compute a^x mod N'''
        current_a = pow(a, 1 << (2 * n - 1 - k), N) # << is binary left shift
        # one iteration of 1-qubit QPE (Quantum Phase Estimation)
        H | ctrl_qubit
        with Control(eng, ctrl_qubit):
            MultiplyByConstantModN(current_a, N) | x

        # perform inverse QFT --> Rotations conditioned on previous outcomes
        for i in range(k):
            if measurements[i]:
                R(-math.pi/(1 << (k - i))) | ctrl_qubit
        H | ctrl_qubit

        # and measure
        Measure | ctrl_qubit
        eng.flush() # flush all gates and execute measurements
        measurements[k] = int(ctrl_qubit)
        if measurements[k]:
            X | ctrl_qubit # reset

        if verbose:
            print("\033[95m{}\033[0m".format(measurements[k]), end="")
            sys.stdout.flush()

    All(Measure) | x
    # turn the measured values into a number in [0,1)
    y = sum([(measurements[2 * n - 1 - i]*1. / (1 << (i + 1)))
             for i in range(2 * n)])

    # continued fraction expansion to get denominator (the period?)
    r = Fraction(y).limit_denominator(N-1).denominator

    # return the (potential) period
    return r

In [27]:
shor(15, 3)

(Note: This is the (slow) Python simulator.)
-----------------------------------

Attempt #0
Random x between 0 and N-1 -->  0

Factors found classically, re-attempt...
-----------------------------------

Attempt #1
Random x between 0 and N-1 -->  2

Period found: 2

Factors found: 3 * 5 = 15.
